In [1]:
import requests
import json
import os
import pickle
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import re
from typing import Dict, List, Optional, Any, Set, Tuple
from dataclasses import dataclass, field
from enum import Enum
import numpy as np

In [2]:
def extract_all_chapters(text: str, output_dir: str = "chapters"):
    # 匹配所有“第X章”标题的位置
    pattern = r"(第[一二三四五六七八九十百千零〇两\d]+章)"
    matches = list(re.finditer(pattern, text))

    if not matches:
        raise ValueError("未找到任何章节标题")

    os.makedirs(output_dir, exist_ok=True)

    for i in range(len(matches)):
        start_idx = matches[i].start()
        end_idx = matches[i+1].start() if i+1 < len(matches) else len(text)
        chapter_title = matches[i].group()
        chapter_number = i + 1  # 用自然数编号
        
        chapter_text = text[start_idx:end_idx].strip()
        filename = os.path.join(output_dir, f"chapter{chapter_number}.txt")
        with open(filename, "w", encoding="utf-8") as f:
            f.write(chapter_text)
        print(f"✅ 已保存：{filename}（{chapter_title}）")

# 读取整本小说
with open("天龙八部.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

# 提取并保存所有章节
extract_all_chapters(full_text)

✅ 已保存：chapters/chapter1.txt（第一章）
✅ 已保存：chapters/chapter2.txt（第二章）
✅ 已保存：chapters/chapter3.txt（第三章）
✅ 已保存：chapters/chapter4.txt（第四章）
✅ 已保存：chapters/chapter5.txt（第五章）
✅ 已保存：chapters/chapter6.txt（第六章）
✅ 已保存：chapters/chapter7.txt（第七章）
✅ 已保存：chapters/chapter8.txt（第八章）
✅ 已保存：chapters/chapter9.txt（第九章）
✅ 已保存：chapters/chapter10.txt（第十章）
✅ 已保存：chapters/chapter11.txt（第十一章）
✅ 已保存：chapters/chapter12.txt（第十二章）
✅ 已保存：chapters/chapter13.txt（第十三章）
✅ 已保存：chapters/chapter14.txt（第十四章）
✅ 已保存：chapters/chapter15.txt（第十五章）
✅ 已保存：chapters/chapter16.txt（第十六章）
✅ 已保存：chapters/chapter17.txt（第十七章）
✅ 已保存：chapters/chapter18.txt（第十八章）
✅ 已保存：chapters/chapter19.txt（第十八章）
✅ 已保存：chapters/chapter20.txt（第十九章）
✅ 已保存：chapters/chapter21.txt（第二十章）
✅ 已保存：chapters/chapter22.txt（第二十一章）
✅ 已保存：chapters/chapter23.txt（第二十二章）
✅ 已保存：chapters/chapter24.txt（第二十三章）
✅ 已保存：chapters/chapter25.txt（第二十四章）
✅ 已保存：chapters/chapter26.txt（第二十五章）
✅ 已保存：chapters/chapter27.txt（第二十六章）
✅ 已保存：chapters/chapter28.txt（第二十七章）
✅ 已保存：chapters/chapter29.txt（第二十

In [3]:
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
os.environ['HUGGINGFACE_HUB_URL'] = 'https://hf-mirror.com' 
os.environ['HF_HUB_BASE_URL'] = 'https://hf-mirror.com'

In [4]:
try:
    from json_repair import repair_json
    HAS_JSONREPAIR = True
    print("✓ jsonrepair库已加载，JSON修复功能已启用")
except ImportError:
    HAS_JSONREPAIR = False
    print("⚠ jsonrepair库未安装，将使用基础修复策略。运行 'pip install jsonrepair' 启用高级JSON修复")
    def repair_json(text):
        return text

⚠ jsonrepair库未安装，将使用基础修复策略。运行 'pip install jsonrepair' 启用高级JSON修复


In [5]:
class TaskType(Enum):
    EVENT_EXTRACTION =  "event_extraction"

In [6]:
class APIClient:
    """API调用客户端 - 支持完整模型配置"""
    
    def __init__(self, api_url: str, api_key: str, 
                 event_model: str = "gpt-4o"):
        self.api_url = api_url
        self.api_key = api_key
        self.event_model = event_model              # 块分析+实体抽取使用的模型
        
        # 创建session并配置连接池和重试策略
        self.session = requests.Session()
        
        retry_strategy = Retry(
            total=3,
            backoff_factor=1,
            status_forcelist=[500, 502, 503, 504],
        )
        
        adapter = HTTPAdapter(pool_connections=10, pool_maxsize=20, max_retries=retry_strategy)
        self.session.mount('http://', adapter)
        self.session.mount('https://', adapter)
        
        self.session.headers.update({
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        })
    
    def call_api(self, messages: List[Dict], task_type: TaskType, timeout: int = 1800) -> Dict:
        """统一的API调用方法 - 根据任务类型选择模型"""
        # 根据任务类型选择模型
        if task_type == TaskType.EVENT_EXTRACTION:
            model_name = self.event_model
        else:
            model_name = self.event_model  # 默认
            
        data = {
            "model": model_name,
            "messages": messages,
            "stream": False
        }
        
        try:
            response = self.session.post(url=self.api_url, json=data, timeout=timeout)
            response.raise_for_status()
            result = response.json()
            return {
                "status": "success",
                "content": result['choices'][0]['message']['content'],
                "model_used": model_name
            }
        except requests.exceptions.RequestException as e:
            return {
                "status": "error",
                "error": str(e),
                "model_used": model_name
            }



In [7]:
def build_event_extraction_prompt(text: str):
    """构造 GPT 对话式抽取事件的 prompt"""
    return [
        {
            "role": "system",
            "content": (
                "你是一个小说事件抽取助手。请从给定段落中提取以下字段："
                "- name（涉及人物）\n"
                "- first_appearance（是否首次出现）\n"
                "- time（时间描述）\n"
                "- location（地点）\n"
                "- action（人物做了什么）\n"
                "- involved_entities（其他涉及对象）\n"
                "- impact（行为产生了什么影响）\n"
                "- utterances（人物说了什么）\n"
                "请返回标准 JSON 格式"
            )
        },
        {
            "role": "user",
            "content": f"以下是小说片段：\n{text}\n\n请返回结构化 JSON 信息："
        }
    ]

In [8]:
api_client = APIClient(
        api_url="http://123.129.219.111:3000/v1/chat/completions",
        api_key="sk-ZXk4KtKH99yelVZKPrpGB2t8OrhUOoVV0hhGi26zgjkN58Ye",
    )

In [ ]:
class Prompt:
    @staticmethod
    def extract_character_names_prompt(paragraph: str, alias_to_name: dict = None):
        system_msg = (
            "你是一个小说人物识别专家，请从以下小说片段中提取所有明确提及的人物。\n"
            "对于每个人物，请标注该人物的标准姓名（如“乔峰”）以及其在该片段中出现的所有称呼、别名、代称（如“丐帮帮主”、“乔帮主”、“那大汉”）。\n\n"
            "请以如下格式返回 JSON：\n"
            "[\n"
            "  {\n"
            "    \"name\": \"乔峰\",\n"
            "    \"aliases\": [\"丐帮帮主\", \"乔帮主\", \"那大汉\"]\n"
            "  }\n"
            "]\n\n"
            "⚠️ 注意：\n"
            "1. 只包含人物，不包括地点或组织。\n"
            "2. 同一人物的多个称呼应统一归并在同一个条目中。\n"
            "3. 所有字段使用标准 JSON 格式。不要包含 markdown 符号或注释。\n"
            "4. 如果无法确定某个称呼是否为新人物，可以暂时保留为独立项。"
        )

        if alias_to_name:
            system_msg += "\n\n以下是已知别名对应的标准人物姓名，请尽量将新识别到的称呼归入已有人物中：\n"
            alias_map_str = json.dumps(alias_to_name, ensure_ascii=False, indent=2)
            system_msg += alias_map_str

        return [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": f"小说片段如下：\n{paragraph}"}
        ]
    @staticmethod
    def update_character_prompt(character_name: str, unfinished_events: list, paragraph: str):
        return [
            {
                "role": "system",
                "content": (
                    "你是小说人物建模专家，将分析某人物的未完成事件与最新小说片段。\n"
                    "你的任务是更新以下字段：\n"
                    "- events：事件列表（更新状态、添加新事件，包含子字段 event_id、action、motivation、impact、involved_entities、time、location、event、if_completed）\n"
                    '''+ - 每个事件必须包含唯一的 "event_id"，例如 "event_001"、"event_002" 等。'''
                    "- utterances：说过的话（含时间或事件编号）\n"
                    "- speech_style：说话风格（如 古典、直接、讽刺等）\n"
                    "- personality_traits：性格（如 冷静、冲动）\n"
                    "- emotion_state：当前情绪状态\n"
                    "- relations：与他人的关系列表（见结构）\n\n"
                    "请特别注意以下要求：\n"
                    "1. 请认真判断现有未完成事件是否已经在新片段中结束。\n"
                    "2.  如果某事件已有结局或结果，请务必将其 `if_completed` 字段标记为 true。\n"
                    "3. 如果小说片段中出现与该人物相关的新事件，请添加新的事件条目。\n"
                    "最终请输出以下 JSON 结构：\n"
                    "{\n"
                    "  \"events\": [...],\n"
                    "  \"utterances\": [...],\n"
                    "  \"speech_style\": \"...\",\n"
                    "  \"personality_traits\": [...],\n"
                    "  \"emotion_state\": \"...\",\n"
                    "  \"relations\": [...]\n"
                    "}\n\n"
                    "⚠️ 请注意：\n"
                    "1. 所有字段名必须使用双引号包裹（JSON 标准格式）。\n"
                    "2. 不要添加注释符号、额外说明或 markdown 符号（如 ```json、// 等）。\n"
                    "3. 仅返回完整 JSON 对象，不能是数组或其他格式。\n"
                    "4. 如没有内容可填，请使用空数组 [] 或空字符串 \"\"。\n"
                )
            },
            {
                "role": "user",
                "content": (
                    f"人物姓名：{character_name}\n"
                    f"当前未完结事件如下（JSON）：\n{json.dumps(unfinished_events, ensure_ascii=False, indent=2)}\n\n"
                    f"小说片段如下：\n{paragraph}\n\n"
                    "请按上述格式返回该人物的更新信息。"
                )
            }
        ]

    @staticmethod
    def speculate_event_outcome(character_name: str, memcube: dict, user_input: str):
        return [
            {
                "role": "system",
                "content": (
                    "你是一位小说剧本推演专家。\n"
                    "你将获得所有人物的完整 JSON 信息（包括事件链、性格、情绪、关系等）以及一条用户提出的假设性情节。\n"
                    "你的任务是基于这些人物的背景、未完成事件、关系网、性格与动机，合理地推演故事可能的发展。\n"
                    "请生成完整的小说段落风格的叙述（不是列表、不是 JSON），描写故事如何展开。\n"
                    "注意语言风格应与原小说保持一致（如古典武侠风）。"
                )
            },
            {
                "role": "user",
                "content": (
                    f"人物姓名：{character_name}\n\n"
                    f"人物信息如下（JSON 格式）：\n{json.dumps(memcube, ensure_ascii=False, indent=2)}\n\n"
                    f"用户的假设性情节如下：\n{user_input}\n\n"
                    "请基于上述信息推演故事的发展，返回小说式语言的叙述，不要包含任何解释性语言或 JSON。"
                )
            }
        ]
    
    @staticmethod
    def evaluate_plot_reasonableness(character_name: str, memcube: dict, user_input: str):
        return [
            {
                "role": "system",
                "content": (
                    "你是一个小说人物行为合理性分析专家。\n"
                    "你将获得所有人物的完整 JSON 信息（包括事件链、性格、情绪、关系等）以及用户提出的一条假设性剧情。\n"
                    "你的任务是：\n"
                    "1. 判断该剧情是否符合该人物的行为逻辑、性格特征、情绪状态以及当前背景。\n"
                    "2. 如不合理，请指出具体不合理的地方，并说明原因。\n"
                    "3. 如合理，请说明其合理性，并简要描述该剧情如何顺理成章地发生。\n\n"
                    "返回格式：\n"
                    "- 合理性评估：合理 / 不合理 / 有条件合理\n"
                    "- 分析说明：详细解释是否符合人物动机、关系与背景\n"
                    "- 建议：如有必要，提出修改建议或更合理的替代表达\n\n"
                    "请用简洁中文回答，不要生成小说正文或 JSON 结构。"
                )
            },
            {
                "role": "user",
                "content": (
                    f"人物姓名：{character_name}\n\n"
                    f"所有人物的完整信息如下（JSON 格式）：\n{json.dumps(memcube, ensure_ascii=False, indent=2)}\n\n"
                    f"用户提出的剧情设想如下：\n{user_input}\n\n"
                    "请你判断这个剧情是否符合该人物当前的状态与逻辑，并说明理由。"
                )
            }
        ]

@staticmethod
def emotion_trajectory_prompt(character_name: str, memcube: dict, user_input: str):
    return [
        {
            "role": "system",
            "content": (
                "你是一个小说人物情绪轨迹分析专家。\n"
                "你将获得某个人物的完整信息（包括事件、性格、情绪、关系等）和用户设想的一段剧情（可能尚未出现在原文中）。\n"
                "请判断在该剧情中，该人物的情绪是否会发生变化。\n\n"
                "你的任务是：\n"
                "1. 判断该剧情设想中是否包含情绪变化。\n"
                "2. 如果有，请指出情绪类型，并解释该变化是如何被激发的。\n"
                "3. 如果没有，请说明为何情绪保持稳定。\n\n"
                "返回格式：\n"
                "- 情绪变化：有 / 无\n"
                "- 当前情绪：xxx\n"
                "- 变化原因：xxx\n"
                "请使用简洁中文作答。"
            )
        },
        {
            "role": "user",
            "content": (
                f"人物姓名：{character_name}\n\n"
                f"该人物的完整信息如下（JSON）：\n{json.dumps(memcube, ensure_ascii=False, indent=2)}\n\n"
                f"用户设想剧情如下：\n{user_input}"
            )
        }
    ]

@staticmethod
def conflict_progression_prompt(character_name: str, memcube: dict, user_input: str):
    return [
        {
            "role": "system",
            "content": (
                "你是一个小说人物之间矛盾关系演化的分析专家。\n"
                "你将获得某人物的完整资料（JSON 格式）以及用户提出的一条设想剧情。\n"
                "请判断在这条剧情中，是否涉及与他人之间的矛盾进展。\n\n"
                "你的任务是：\n"
                "1. 判断该设想剧情中是否涉及已有或潜在冲突对象。\n"
                "2. 如果有，请判断该关系是否发生变化（如激化、缓和或解决）。\n"
                "3. 简述冲突变化的原因。\n\n"
                "返回格式：\n"
                "- 对手：xxx\n"
                "- 当前阶段：xxx（如：潜在 → 升级 → 缓和 → 解决）\n"
                "- 变化原因：xxx\n"
                "请使用简洁中文作答。"
            )
        },
        {
            "role": "user",
            "content": (
                f"人物姓名：{character_name}\n\n"
                f"该人物的完整信息如下（JSON）：\n{json.dumps(memcube, ensure_ascii=False, indent=2)}\n\n"
                f"用户设想剧情如下：\n{user_input}"
            )
        }
    ]

@staticmethod
def character_position_prompt(character_name: str, memcube: dict, user_input: str):
    return [
        {
            "role": "system",
            "content": (
                "你是一个小说人物立场判断专家。\n"
                "你将获得某角色的完整 JSON 信息和一段用户设想的剧情。\n"
                "你的任务是：\n"
                "1. 判断该人物是否在剧情中对某个议题表达了明确立场（如支持、反对、中立、困惑）。\n"
                "2. 如果表达了，请指出该议题和该人物的态度，并解释是否符合他当前的行为逻辑和背景。\n\n"
                "返回格式：\n"
                "- 议题：xxx\n"
                "- 立场：支持 / 反对 / 中立 / 困惑\n"
                "- 原因：xxx\n"
                "请使用简洁中文作答。"
            )
        },
        {
            "role": "user",
            "content": (
                f"人物姓名：{character_name}\n\n"
                f"该人物的完整信息如下（JSON）：\n{json.dumps(memcube, ensure_ascii=False, indent=2)}\n\n"
                f"用户设想剧情如下：\n{user_input}"
            )
        }
    ]


# ✅ 构建人物事件列表的子集（仅包含未完结事件）
def get_unfinished_events(memcube: dict):
    return [event for event in memcube.get("event_chain", []) if not event.get("if_completed", False)]

# ✅ 合并 GPT 结果到原人物 JSON
from copy import deepcopy

def update_memcube_with_gpt_result(memcube: dict, updated_events: list):
    new_memcube = deepcopy(memcube)
    new_memcube["event_chain"] = updated_events
    return new_memcube


In [11]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def update_memcube_for_character(name, memcube, content, chapter_id):
    try:
        unfinished = get_unfinished_events(memcube)
        update_prompt = Prompt.update_character_prompt(name, unfinished, content)
        update_result = api_client.call_api(update_prompt, "update_character", timeout=1800)

        raw = update_result.get("content", "").strip("```json").strip("```").strip()
        if not raw:
            return name, None, "空返回"

        updated = json.loads(raw)
        return name, updated, None

    except Exception as e:
        return name, None, str(e)

In [ ]:
def get_unfinished_events(memcube: dict):
    return [event for event in memcube.get("events", []) if not event.get("if_completed", False)]

def init_memcube(character_name: str, chunk_id: str):
    return {
        "name": character_name,
        "first_appearance": chunk_id,
        "aliases": [character_name],
        "events": [],
        "utterances": [],
        "speech_style": "",
        "personality_traits": [],
        "emotion_state": "",
        "relations": []
    }

def merge_events(old_events: list, new_events: list):
    """合并事件：新事件覆盖旧事件；若旧事件被更新（如状态变为已完成），则同步更新"""
    event_dict = {e["event_id"]: e for e in old_events}

    for new_event in new_events:
        eid = new_event["event_id"]
        if eid in event_dict:
            # 合并逻辑：新字段优先，保留 event_id
            merged = event_dict[eid].copy()
            for key, value in new_event.items():
                if value not in [None, "", []]:
                    merged[key] = value
            event_dict[eid] = merged
        else:
            event_dict[eid] = new_event  # 新事件直接加入

    return list(event_dict.values())

def merge_unique_list(old: list, new: list):
    """合并两个列表并去重，保持原有顺序"""
    combined = old + new
    seen = set()
    result = []
    for item in combined:
        if isinstance(item, dict):
            key = json.dumps(item, sort_keys=True, ensure_ascii=False)
        else:
            key = str(item)
        if key not in seen:
            seen.add(key)
            result.append(item)
    return result

memcubes = {}
alias_to_name = {}
chapter_folder = "chapters"

chapter_files = sorted(
    [os.path.join(chapter_folder, f) for f in os.listdir(chapter_folder) if f.startswith("chapter") and f.endswith(".txt")],
    key=lambda x: int(re.search(r'chapter(\d+)', x).group(1))
)

for chapter_file in chapter_files:
    chapter_id = chapter_file.replace(".txt", "")
    print(f"\n📖 正在处理：{chapter_id}")

    with open(chapter_file, "r", encoding="utf-8") as f:
        content = f.read()

    name_prompt = Prompt.extract_character_names_prompt(content, alias_to_name)
    name_result = api_client.call_api(name_prompt, "name_extraction", timeout=1800)
    try:
        extracted = json.loads(name_result.get("content", "").strip("```json").strip("```").strip())
    except:
        extracted = []

    # alias 映射表（可用于后续处理）
   

    # 🧱 Step 2: 初始化人物
    for item in extracted:
        std_name = item["name"]
        aliases = item.get("aliases", [])
        
        # 初始化 memcube
        if std_name not in memcubes:
            print(f"🆕 新人物识别：{std_name}")
            memcubes[std_name] = init_memcube(std_name, chapter_id)
            memcubes[std_name]["aliases"] = []  # 初始化 alias 列表

        # 更新 alias
        all_aliases = list(set(memcubes[std_name].get("aliases", []) + aliases))
        memcubes[std_name]["aliases"] = all_aliases

        # 构建 alias 到标准名的映射
        for alias in [std_name] + aliases:
            alias_to_name[alias] = std_name

    # 🔄 Step 3: 更新人物状态
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = {
            executor.submit(update_memcube_for_character, name, memcube, content, chapter_id): name
            for name, memcube in memcubes.items()
        }

        for future in as_completed(futures):
            name = futures[future]
            try:
                name, updated, error = future.result()
                if error or not updated:
                    print(f"⚠️ 更新失败：{name} in {chapter_id} -> {error}")
                    continue

                memcube = memcubes[name]
                memcube["events"] = merge_events(memcube["events"], updated.get("events", []))
                memcube["utterances"].extend(updated.get("utterances", []))
                if updated.get("speech_style"):
                    memcube["speech_style"] = updated["speech_style"]
                memcube["personality_traits"] = merge_unique_list(
                    memcube["personality_traits"], updated.get("personality_traits", [])
                )
                if updated.get("emotion_state"):
                    memcube["emotion_state"] = updated["emotion_state"]
                memcube["relations"].extend(updated.get("relations", []))

            except Exception as e:
                print(f"⚠️ 并行执行异常：{name} -> {e}")

# 💾 保存最终 memcubes
with open("memcubes_all.json", "w", encoding="utf-8") as f:
    json.dump(memcubes, f, ensure_ascii=False, indent=2)

print("\n✅ 所有章节处理完成，总人物数：", len(memcubes))


📖 正在处理：chapters/chapter1
🆕 新人物识别：龚光杰
🆕 新人物识别：褚师弟
🆕 新人物识别：左子穆
🆕 新人物识别：辛双清
🆕 新人物识别：段誉
🆕 新人物识别：马五德
🆕 新人物识别：钟灵
🆕 新人物识别：司空玄

📖 正在处理：chapters/chapter2
🆕 新人物识别：干光豪
🆕 新人物识别：葛师妹
🆕 新人物识别：钟万仇
🆕 新人物识别：钟夫人
🆕 新人物识别：岳老二
🆕 新人物识别：来福儿
⚠️ 更新失败：辛双清 in chapters/chapter2 -> Extra data: line 107 column 1 (char 2233)

📖 正在处理：chapters/chapter3
🆕 新人物识别：瑞婆婆
🆕 新人物识别：平婆婆
🆕 新人物识别：木婉清

📖 正在处理：chapters/chapter4
🆕 新人物识别：南海鳄神
🆕 新人物识别：叶二娘
🆕 新人物识别：云中鹤
🆕 新人物识别：郁光标
🆕 新人物识别：吴光胜

📖 正在处理：chapters/chapter5
🆕 新人物识别：褚万里
🆕 新人物识别：朱丹臣
🆕 新人物识别：古笃诚
🆕 新人物识别：傅思归
🆕 新人物识别：葛光佩

📖 正在处理：chapters/chapter6
🆕 新人物识别：玉虚散人
🆕 新人物识别：段正淳
🆕 新人物识别：高升泰
🆕 新人物识别：段正明
🆕 新人物识别：秦红棉

📖 正在处理：chapters/chapter7
🆕 新人物识别：恶贯满盈
⚠️ 更新失败：叶二娘 in chapters/chapter7 -> Invalid control character at: line 18 column 51 (char 516)
⚠️ 更新失败：云中鹤 in chapters/chapter7 -> Invalid control character at: line 28 column 10 (char 664)

📖 正在处理：chapters/chapter8
🆕 新人物识别：镇南王妃刀白凤
🆕 新人物识别：保定帝段正明
🆕 新人物识别：巴天石
🆕 新人物识别：华赫艮
🆕 新人物识别：范骅
🆕 新人物识别：黄眉僧
🆕 新人物识别：司空见惯
🆕 新人物识别：巴司空

📖 正在处理：chapters/chapter

In [ ]:
with open("memcubes1.json", "r", encoding="utf-8") as f:
    memcubes = json.load(f)

character_name = "段誉"
user_input = "如果段誉没有出现在剑湖宫比武大会，事情会怎样？"

# 构造 prompt
prompt = Prompt.speculate_event_outcome(character_name, memcubes, user_input)

# 调 GPT
response = api_client.call_api(prompt, "speculative_story")
print(response.get("content", "❌ 没有返回"))

剑湖宫中，秋风渐起，树影婆娑。无量山难得的热闹一如既往，宗门弟子已集结于练武厅，东宗与西宗的比剑厮杀，一如锁定七龙珠般的高潮。然而，众人等待的席间却少了段誉这妙人，与平日相比，情势似乎少了些意外的光彩。

褚师弟站在剑湖宫练武厅的中央，感受到众多观战者的目光投射。比武的剑光交错，龚师兄在他身侧气势如虹，剑招犹如急风骤雨，每一击都使得褚师弟心头微颤。他明白此战若败，剑湖宫五年的权力只能由东宗继续掌握。对龚师兄的自信与得意，褚师弟心生怨恨，但他仍在心中默默维护着剑术的尊严。

左子穆轻拍衣袖，面色如常，然而隐藏的怒意依然在心中起伏。原本提议于段誉的比试成了泡影，东宗的龚师兄以玲珑的步法获胜，让无量剑西宗的实力稍显暗淡。左子穆坚毅如斯，暗暗记住了辛师妹的眼中鄙意，心底之火无法压制。

辛师妹在一旁笑声轻蔑，无量玉壁的钻研成果隐隐显现，她对左子穆的腹中有话虽未细表，但东宗下一次比武必然有意。她的弟子褚师弟落败，却仍不失继续努力的决心。

而钟灵则坐在梁柱上，手中把玩着一只闪电貂，目光逼向龚师兄，她的心里有些难以言表的好奇，但转瞬被无量剑的比试吸引住。神农帮未曾在此时出现，练武厅依然是无量剑的主场。但心底明白，她用灵巧身姿也不愿在比武大会上独自一人做主角。

正因段誉未至，习惯于斗剑的马五德继续专注于观战。无量剑的两派竞争虽不免激烈，但并未爆发更多的意外事件。左子穆观战之际，让龚师兄不经意间偷瞄见繁星闪烁，心下虽有些瑕疵，东宗却继续处于优势。

夜幕降临，无量山上剑影依稀，霜寒蓄积，江湖的慷慨和剑湖宫依然如故。在宗派斗争繁琐而复杂的面前，剑湖宫并无段誉的幽默与趣谈，或许明朝霞起，剑湖宫上将卷起另一番风波。


In [18]:
memcubes

{'段誉': {'name': '段誉',
  'first_appearance': 'chapters/chapter1',
  'events': [{'event_id': 'event_003',
    'time': '第十六章',
    'location': '杏子林',
    'event': '段誉在杏子林遭遇丐帮内乱后西夏来犯',
    'if_completed': True,
    'action': ['试图说服乔峰留下', '与王语嫣交流并保护她', '用六脉神剑击退努儿海', '实施凌波微步逃脱'],
    'motivation': ['保护王语嫣不受伤害', '不愿和丐帮直接冲突'],
    'involved_entities': ['乔峰', '王语嫣', '西夏武士', '努儿海', '丐帮'],
    'impact': ['与丐帮的关系变得复杂', '帮助乔峰及王语嫣避险', '自身武功暴露给西夏'],
    'utterances': [{'event_id': 'event_003', 'text': '大哥，大哥，我随你去！'},
     {'event_id': 'event_003', 'text': '王姑娘，你们要到那里去？'}],
    'speech_style': '直接',
    'personality_traits': ['谨慎', '善良', '机智'],
    'emotion_state': '混乱但镇定',
    'relations': [{'target': '乔峰',
      'relation_type': '结拜兄弟',
      'relation_strength': 0.9,
      'last_updated': 'event_002',
      'derived_from': 'action:结义, impact:共同进退'},
     {'target': '王语嫣',
      'relation_type': '朋友',
      'relation_strength': 0.75,
      'last_updated': 'event_003',
      'derived_from': 'action:保

In [32]:
alias_to_name = {}